In [1]:
%load_ext autoreload
%autoreload 2
# you can check out the the documentation for the rest of the autoreaload modes
# by apending a question mark to %autoreload, like this:
# %autoreload?


from IPython.lib import deepreload
import imp
def my_reload(modules=[], deep=False):
    if not isinstance(modules, list):
        modules = [modules]
    if deep:
        rl_func = deepreload.reload
    else:
        rl_func = imp.reload
    
    for m in modules:
        try:
            rl_func(m)
        except:
            print(f"THIS IS A COMMON ISSUE: some modules in {m} are not work")

import sys, os, yaml
from pathlib import Path

sys.path.insert(0,"/Users/bill/Projects/Pycharm_projects/AutoRAG")
config_path = "/Users/bill/Projects/Pycharm_projects/AutoRAG/ragas/configuration/config.yaml"

/var/folders/zy/dvp0hbvx6x7bkqw0xgf9skg00000gn/T/ipykernel_62581/71235559.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
from ragas.configuration import RetrievalConfiguration,VectorDBConfiguration,GeneratorConfiguration

ModuleConfig = VectorDBConfiguration.load_from_yaml(config_path, key="vectordb")

# config_path = "/Users/bill/Projects/Pycharm_projects/AutoRAG/ragas/configuration/config.yaml"
# retriever = RetrievalConfiguration.load_from_yaml(config_path, key="retrieval")

In [3]:
ModuleConfig.cs

Configuration space object:
  Hyperparameters:
vectordb
    collection_name, Type: Categorical, Choices: {huggingface_all_mpnet_base_v2}, Default: huggingface_all_mpnet_base_v2
    db_type, Type: Categorical, Choices: {chroma}, Default: chroma
    embedding_batch, Type: Categorical, Choices: {100, 200}, Default: 100
    embedding_model, Type: Categorical, Choices: {huggingface_all_mpnet_base_v2}, Default: huggingface_all_mpnet_base_v2
    similarity_metric, Type: Categorical, Choices: {cosine, euclidean, ip}, Default: cosine

In [4]:
ModuleConfig.create_nodes(3)

[{'name': 'default_vectordb',
  'collection_name': 'huggingface_all_mpnet_base_v2',
  'db_type': 'chroma',
  'embedding_batch': 200,
  'embedding_model': 'huggingface_all_mpnet_base_v2',
  'similarity_metric': 'cosine',
  'client_type': 'persistent',
  'host': 'localhost',
  'port': 8000,
  'ssl': False,
  'headers': None,
  'api_key': None,
  'tanant': 'default_tenant',
  'database': 'default_database',
  'path': '/home/lyb/RAG/experiments/chroma_mpnet'},
 {'name': 'default_vectordb',
  'collection_name': 'huggingface_all_mpnet_base_v2',
  'db_type': 'chroma',
  'embedding_batch': 100,
  'embedding_model': 'huggingface_all_mpnet_base_v2',
  'similarity_metric': 'cosine',
  'client_type': 'persistent',
  'host': 'localhost',
  'port': 8000,
  'ssl': False,
  'headers': None,
  'api_key': None,
  'tanant': 'default_tenant',
  'database': 'default_database',
  'path': '/home/lyb/RAG/experiments/chroma_mpnet'},
 {'name': 'default_vectordb',
  'collection_name': 'huggingface_all_mpnet_base

In [102]:
from ragas.configuration import *
from pathlib import Path
import yaml


size = 3
retriever = RetrievalConfiguration.load_from_yaml(config_path, key="retrieval")
retriever_node_lines = retriever.create_node_lines(size=size)

vectordb = VectorDBConfiguration.load_from_yaml(config_path, key="vectordb")
vectordb_node_lines = vectordb.create_node_lines(size=size)

generator = GeneratorConfiguration.load_from_yaml(config_path, key="generator")
generator_nodes = generator.create_nodes(size=size)

prompt_maker = PromptMakerConfiguration.load_from_yaml(config_path, key="prompt_maker")
prompt_maker_nodes = prompt_maker.create_nodes(size=size)

post_retrieve_node_lines = create_lines_with_nodes("post_retrieve_node_line", prompt_maker_nodes, generator_nodes)

zipped = zip(vectordb_node_lines, retriever_node_lines, post_retrieve_node_lines)
for i, node_line in enumerate(zipped):
    vectordb_node_line, retriever_node_line, generator_node_line = node_line
    print(node_line)

    data = {
        'vectordb': [
            vectordb_node_line
        ],
        'node_lines': [
            retriever_node_line,
            generator_node_line
        ],
    }

    # Save the modified data back to a YAML file
    with open(f"../candidate_config/ollama_config_{i}.yaml", "w") as file:
        yaml.safe_dump(data, file, default_flow_style=False, sort_keys=False)

{'method': ['hybrid_rrf'], 'top_k': [5, 10], 'bm25': {'bm25_tokenizer': ['porter_stemmer', 'space']}, 'vectordb': {'embedding_batch': [256]}, 'hybrid_cc': {'normalize_method': ['tmm', 'mm', 'z', 'dbsf'], 'weight': (0.0, 1.0), 'lexical_theoretical_min_value': 0, 'semantic_theoretical_min_value': -1}, 'hybrid_rrf': {'weight': (4, 80)}}
{'method': ['chroma'], 'collection_name': ['huggingface_all_mpnet_base_v2'], 'embedding_model': ['huggingface_all_mpnet_base_v2'], 'embedding_batch': [100, 200], 'similarity_metric': ['cosine', 'l2', 'ip'], 'couchbase': {'ingest_batch': [100, 200]}, 'pinecone': {'ingest_batch': [100, 200]}, 'milvus': {'ingest_batch': [100, 200], 'index_type': ['IVF_FLAT', 'ScaNN', 'IVF_SQ8', 'SCANN', 'GPU_CAGRA', 'GPU_IVF_FLAT']}}
{'method': ['LlamaIndexLLM'], 'temperature': [0.1, 0.5], 'batch': [1], 'max_token': 512, 'LlamaIndexLLM': {'llm': ['ollama'], 'model': ['deepseek-r1:70b'], 'torch_dtype': 'float16'}, 'OpenAILLM': {'llm': ['gpt-3.5-turbo', 'gpt-4-turbo-2024-04-09'

In [99]:
from ConfigSpace.util import generate_grid

s = generate_grid(retriever.cs)

ValueError: num_steps_dict is None or doesn't contain the number of points to divide [hybrid_cc]weight into. And its quantization factor is None. Please provide/set one of these values.

In [106]:
retriever.cs.get('[bm25]bm25_tokenizer') == None

True

In [98]:
result_df = pd.DataFrame(
            {
                "filename": [1],
                "execution_time": [2],
                "average_output_token": [3],
            }
        )

import fcntl  # Use portalocker for Windows
def write_summary(df_path="summary.csv", df: pd.DataFrame=None):
    """Safely appends a DataFrame to summary.csv using file locking."""
    with open(df_path, "a") as f:
        try:
            fcntl.flock(f, fcntl.LOCK_EX)  # Lock the file for exclusive access
            df.to_csv(f, header=True, index=False)  # Append new rows without headers
            f.flush()  # Ensure data is written immediately
        finally:
            fcntl.flock(f, fcntl.LOCK_UN)  # Unlock the file

df_path="summary.csv"
write_summary(df=result_df)

In [93]:
result_df.to_csv(df_path, index=False)

In [41]:
retriever.config

{'top_k': [5, 10, 20, 30]}

In [ ]:
def create_lines_with_nodes(self, *nodes) -> List[Dict]:
    node_lines = [{
        "node_line_name": "post_retrieve_node_line",
        "nodes": list(node_group)
    } for node_group in zip(*nodes)]
    return node_lines

# test create_lines_with_nodes
nodes = []


## Hand-crafted YAML Creation

In [12]:
import yaml

with open("../ollama_config.yaml", "r") as file:
    data = yaml.safe_load(file)

print("Loaded YAML Data:")

data

Loaded YAML Data:


{'vectordb': [{'name': 'chroma_mpnet',
   'db_type': 'chroma',
   'client_type': 'persistent',
   'collection_name': 'huggingface_all_mpnet_base_v2',
   'embedding_model': 'huggingface_all_mpnet_base_v2',
   'path': '/home/lyb/RAG/experiments/chroma_mpnet'}],
 'node_lines': [{'node_line_name': 'retrieve_node_line',
   'nodes': [{'node_type': 'retrieval',
     'strategy': {'metrics': ['retrieval_f1',
       'retrieval_recall',
       'retrieval_precision']},
     'top_k': 3,
     'modules': [{'module_type': 'bm25'},
      {'module_type': 'vectordb', 'vectordb': 'chroma_mpnet'},
      {'module_type': 'hybrid_rrf',
       'target_modules': "('bm25', 'vectordb')",
       'rrf_k': [3, 5, 10]},
      {'module_type': 'hybrid_cc',
       'target_modules': "('bm25', 'vectordb')",
       'weights': '(0.5, 0.5)'}]}]},
  {'node_line_name': 'post_retrieve_node_line',
   'nodes': [{'node_type': 'prompt_maker',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']},
     'modules': [{'module